In [48]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
import pathway as pw
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import layout
from bokeh.palettes import Category20

# Dynamic Pricing for Urban Parking Lots

This notebook implements **Model 1 (Baseline Linear Model)** for dynamic pricing based on parking occupancy.

- Base Price: $10
- Price increases linearly with occupancy
- Implemented using Pandas & NumPy only (per constraints)

---


In [49]:
# Loading the dataset
df=pd.read_csv('dataset.csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [50]:
# Combine date and time into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Encode TrafficConditionNearby
traffic_mapping = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_mapping)

# Encode VehicleType
vehicle_mapping = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_mapping)

# Sort by parking lot and timestamp
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)


df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,TrafficLevel,VehicleWeight
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1.0,1.0
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00,1.0,1.0
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00,1.0,1.0
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00,1.0,1.0
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00,1.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00,2016-12-19 14:30:00,NaN,1.5
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00,2016-12-19 15:03:00,1.0,1.0
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00,2016-12-19 15:29:00,1.0,NaN
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00,2016-12-19 16:03:00,1.0,1.0


In [51]:
# Set base price and alpha
BASE_PRICE = 10.0
ALPHA = 1.0

# Initialize new column
df['Price_Model1'] = np.nan

# Compute price per lot
for lot in df['SystemCodeNumber'].unique():
    lot_mask = df['SystemCodeNumber'] == lot
    lot_data = df[lot_mask]
    prices = [BASE_PRICE]

    for i in range(1, len(lot_data)):
        prev_price = prices[-1]
        occupancy_ratio = lot_data.iloc[i-1]['Occupancy'] / lot_data.iloc[i-1]['Capacity']
        new_price = prev_price + ALPHA * occupancy_ratio
        prices.append(new_price)

    df.loc[lot_mask, 'Price_Model1'] = prices

# Display sample results
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'Price_Model1']].head(10)


,SystemCodeNumber,Timestamp,Occupancy,Capacity,Price_Model1
0,BHMBCCMKT01,2016-10-04 07:59:00,61,577,10.000000
1,BHMBCCMKT01,2016-10-04 08:25:00,64,577,10.105719
2,BHMBCCMKT01,2016-10-04 08:59:00,80,577,10.216638
3,BHMBCCMKT01,2016-10-04 09:32:00,107,577,10.355286
4,BHMBCCMKT01,2016-10-04 09:59:00,150,577,10.540728
5,BHMBCCMKT01,2016-10-04 10:26:00,177,577,10.800693
6,BHMBCCMKT01,2016-10-04 10:59:00,219,577,11.107452
7,BHMBCCMKT01,2016-10-04 11:25:00,247,577,11.487002
8,BHMBCCMKT01,2016-10-04 11:59:00,259,577,11.915078
9,BHMBCCMKT01,2016-10-04 12:29:00,266,577,12.363951


## ✅ Model 2: Demand-Based Dynamic Pricing

This model uses a weighted demand function based on:

- Occupancy rate
- Queue length
- Traffic congestion
- Special day indicator
- Vehicle type weight

### Price Formula:
Demand Equation

Demand = α × (Occupancy / Capacity) + β × QueueLength - γ × TrafficLevel + δ × IsSpecialDay + ε × VehicleWeight

Price Equation

Price(t) = BasePrice × (1 + λ × NormalizedDemand)

- Price is bounded between **$5 and $20**
- Demand is normalized across all data points


In [52]:
# Hyperparameters (can be tuned)
alpha = 1.0   # weight for occupancy
beta = 0.5    # queue length
gamma = 0.4   # traffic level
delta = 1.5   # special day
epsilon = 0.75 # vehicle type weight
lambda_ = 1.0  # demand impact on price

# Calculate raw demand
df['RawDemand'] = (
    alpha * (df['Occupancy'] / df['Capacity']) +
    beta * df['QueueLength'] -
    gamma * df['TrafficLevel'] +
    delta * df['IsSpecialDay'] +
    epsilon * df['VehicleWeight']
)

# Normalize demand between 0 and 1
min_demand = df['RawDemand'].min()
max_demand = df['RawDemand'].max()
df['NormalizedDemand'] = (df['RawDemand'] - min_demand) / (max_demand - min_demand)

# Calculate price using normalized demand
df['Price_Model2'] = 10 * (1 + lambda_ * df['NormalizedDemand'])

# Clip price between $5 and $20
df['Price_Model2'] = df['Price_Model2'].clip(lower=5, upper=20)

# View sample
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength', 'TrafficLevel',
    'IsSpecialDay', 'VehicleWeight', 'Price_Model2']].head(10)


,SystemCodeNumber,Timestamp,Occupancy,QueueLength,TrafficLevel,IsSpecialDay,VehicleWeight,Price_Model2
0,BHMBCCMKT01,2016-10-04 07:59:00,61,1,1.0,0,1.0,11.018170
1,BHMBCCMKT01,2016-10-04 08:25:00,64,1,1.0,0,1.0,11.023675
2,BHMBCCMKT01,2016-10-04 08:59:00,80,2,1.0,0,1.0,11.582418
3,BHMBCCMKT01,2016-10-04 09:32:00,107,2,1.0,0,1.0,11.631962
4,BHMBCCMKT01,2016-10-04 09:59:00,150,2,1.0,0,0.5,11.313827
5,BHMBCCMKT01,2016-10-04 10:26:00,177,3,1.0,0,1.0,12.289792
6,BHMBCCMKT01,2016-10-04 10:59:00,219,6,3.0,0,1.5,13.505036
7,BHMBCCMKT01,2016-10-04 11:25:00,247,5,NaN,0,1.0,NaN
8,BHMBCCMKT01,2016-10-04 11:59:00,259,5,NaN,0,NaN,NaN
9,BHMBCCMKT01,2016-10-04 12:29:00,266,8,3.0,0,0.5,13.855971


## ✅ Model 3: Competitive Pricing Based on Nearby Lots

This model factors in competitor pricing using location intelligence.

### Logic:
- If a lot is full and nearby lots are cheaper → suggest rerouting or lower price.
- If nearby lots are expensive → current lot price can increase.
- Distance between lots is computed using **Haversine formula**.

This encourages smarter pricing in competitive zones.

---


In [53]:
from math import radians, sin, cos, sqrt, atan2

# Compute haversine distance (in km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c


In [54]:
# Precompute parking lot locations
lot_locations = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first().to_dict('index')

# Set radius to consider nearby lots (in km)
RADIUS_KM = 1.0
base_price = 10

# Create placeholder column
df['Price_Model3'] = df['Price_Model2']  # Start from demand-based price

# Go through each row and adjust based on competitor proximity
for i, row in df.iterrows():
    lot_id = row['SystemCodeNumber']
    lat1, lon1 = lot_locations[lot_id]['Latitude'], lot_locations[lot_id]['Longitude']

    # Find competitor lots within radius
    competitor_prices = []
    for other_id, coords in lot_locations.items():
        if other_id == lot_id:
            continue
        dist = haversine(lat1, lon1, coords['Latitude'], coords['Longitude'])
        if dist <= RADIUS_KM:
            # Get last known price of competitor at that time
            time_match = df[(df['SystemCodeNumber'] == other_id) & (df['Timestamp'] == row['Timestamp'])]
            if not time_match.empty:
                competitor_prices.append(time_match['Price_Model2'].values[0])

    if competitor_prices:
        avg_competitor_price = np.mean(competitor_prices)

        # If current lot is full and competitors are cheaper
        if row['Occupancy'] >= row['Capacity'] and avg_competitor_price < row['Price_Model2']:
            df.at[i, 'Price_Model3'] = max(row['Price_Model2'] * 0.95, 5)  # slight discount
        # If competitors are more expensive
        elif avg_competitor_price > row['Price_Model2']:
            df.at[i, 'Price_Model3'] = min(row['Price_Model2'] * 1.05, 20)  # slight increase
# Display sample results
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Price_Model1', 'Price_Model2', 'Price_Model3']].head(10)



,SystemCodeNumber,Timestamp,Occupancy,Price_Model1,Price_Model2,Price_Model3
0,BHMBCCMKT01,2016-10-04 07:59:00,61,10.000000,11.018170,11.018170
1,BHMBCCMKT01,2016-10-04 08:25:00,64,10.105719,11.023675,11.023675
2,BHMBCCMKT01,2016-10-04 08:59:00,80,10.216638,11.582418,11.582418
3,BHMBCCMKT01,2016-10-04 09:32:00,107,10.355286,11.631962,11.631962
4,BHMBCCMKT01,2016-10-04 09:59:00,150,10.540728,11.313827,11.313827
5,BHMBCCMKT01,2016-10-04 10:26:00,177,10.800693,12.289792,12.289792
6,BHMBCCMKT01,2016-10-04 10:59:00,219,11.107452,13.505036,13.505036
7,BHMBCCMKT01,2016-10-04 11:25:00,247,11.487002,NaN,NaN
8,BHMBCCMKT01,2016-10-04 11:59:00,259,11.915078,NaN,NaN
9,BHMBCCMKT01,2016-10-04 12:29:00,266,12.363951,13.855971,13.855971


## 🚗 Rerouting Recommendation System

When a parking lot is **full** or **nearly full**, and there are **nearby lots** that:
- Have available capacity
- Offer a lower or similar price

→ Recommend rerouting the vehicle there.

We use:
- Distance threshold: 1 km
- Price comparison using Model 3


In [55]:
# Define rerouting suggestion column
df['RerouteTo'] = None
distance_threshold_km = 1.0

for i, row in df.iterrows():
    if row['Occupancy'] >= row['Capacity']:
        current_lot = row['SystemCodeNumber']
        current_time = row['Timestamp']
        lat1, lon1 = lot_locations[current_lot]['Latitude'], lot_locations[current_lot]['Longitude']
        current_price = row['Price_Model3']

        # Search for alternative lots
        candidates = []
        for other_lot, coords in lot_locations.items():
            if other_lot == current_lot:
                continue
            dist = haversine(lat1, lon1, coords['Latitude'], coords['Longitude'])
            if dist <= distance_threshold_km:
                # Get the state of the other lot at the same timestamp
                other_row = df[(df['SystemCodeNumber'] == other_lot) & (df['Timestamp'] == current_time)]
                if not other_row.empty:
                    other_row = other_row.iloc[0]
                    if other_row['Occupancy'] < other_row['Capacity'] and other_row['Price_Model3'] <= current_price:
                        candidates.append((other_lot, other_row['Price_Model3'], dist))

        # Recommend the best candidate (lowest price, then nearest)
        if candidates:
            best = sorted(candidates, key=lambda x: (x[1], x[2]))[0]
            df.at[i, 'RerouteTo'] = best[0]


In [56]:
# Show entries where rerouting is suggested
df[df['RerouteTo'].notna()][['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'Price_Model3', 'RerouteTo']].head(10)


,SystemCodeNumber,Timestamp,Occupancy,Capacity,Price_Model3,RerouteTo
2076,BHMBCCTHL01,2016-11-17 12:04:00,390,387,14.434849,BHMBCCMKT01
2078,BHMBCCTHL01,2016-11-17 13:04:00,390,387,14.302503,BHMBCCMKT01
2079,BHMBCCTHL01,2016-11-17 13:31:00,394,387,14.313446,BHMBCCMKT01
2112,BHMBCCTHL01,2016-11-19 12:04:00,402,387,15.261755,BHMEURBRD01
2113,BHMBCCTHL01,2016-11-19 12:31:00,402,387,14.864717,BHMEURBRD01
2114,BHMBCCTHL01,2016-11-19 13:04:00,399,387,13.930087,Others-CCCPS119a
2115,BHMBCCTHL01,2016-11-19 13:31:00,402,387,14.335333,Others-CCCPS119a
2118,BHMBCCTHL01,2016-11-19 15:04:00,388,387,12.497125,Others-CCCPS202
2119,BHMBCCTHL01,2016-11-19 15:31:00,388,387,12.497125,Others-CCCPS119a
2121,BHMBCCTHL01,2016-11-19 16:31:00,402,387,12.535427,Others-CCCPS119a


In [57]:


# Combine date and time into one datetime column
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Encode traffic levels
df['traffic_level'] = df['TrafficConditionNearby'].map({'low': 1, 'medium': 2, 'high': 3})

# Encode vehicle types
df['vehicle_weight'] = df['VehicleType'].map({'bike': 0.5, 'car': 1.0, 'truck': 1.5})

# Create daily date column
df['date'] = df['timestamp'].dt.date

# --- Step 2: Group by lot and date
daily = df.groupby(['SystemCodeNumber', 'date']).agg({
    'Occupancy': 'mean',
    'Capacity': 'mean',
    'QueueLength': 'mean',
    'traffic_level': 'mean',
    'IsSpecialDay': 'max',
    'vehicle_weight': 'mean'
}).reset_index()

# --- Step 3: Compute dynamic price
alpha, beta, gamma, delta, epsilon, lambda_ = 1.0, 0.5, 0.4, 1.5, 0.75, 1.0
base_price = 10

daily['demand'] = (
    alpha * (daily['Occupancy'] / daily['Capacity']) +
    beta * daily['QueueLength'] -
    gamma * daily['traffic_level'] +
    delta * daily['IsSpecialDay'] +
    epsilon * daily['vehicle_weight']
)

# Normalize demand
min_d, max_d = daily['demand'].min(), daily['demand'].max()
daily['normalized_demand'] = (daily['demand'] - min_d) / (max_d - min_d)

# Calculate final price
daily['dynamic_price'] = base_price * (1 + lambda_ * daily['normalized_demand'])
daily['dynamic_price'] = daily['dynamic_price'].clip(lower=5, upper=20)

# --- Step 4: Visualization with Bokeh
# Ensure datetime type for plotting
daily['date'] = pd.to_datetime(daily['date'])

# Create Bokeh figure
p = figure(
    title="📈 Daily Dynamic Pricing per Parking Lot",
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label="Price ($)",
    width=950,
    height=500,
    tools="pan,wheel_zoom,box_zoom,reset"
)

# Add hover tool
hover = HoverTool(
    tooltips=[
        ("Lot", "@system_code_number"),
        ("Date", "@date{%F}"),
        ("Price", "@dynamic_price{$0.00}")
    ],
    formatters={'@date': 'datetime'},
    mode='vline'
)
p.add_tools(hover)

# Plot per lot
unique_lots = daily['SystemCodeNumber'].unique()
colors = Category10[min(10, len(unique_lots))]

for i, lot in enumerate(unique_lots):
    lot_df = daily[daily['SystemCodeNumber'] == lot].copy()
    lot_df = lot_df.rename(columns={'SystemCodeNumber': 'system_code_number'})
    source = ColumnDataSource(lot_df[['date', 'dynamic_price', 'system_code_number']])
    p.line('date', 'dynamic_price', source=source,
           legend_label=lot, line_width=2,
           color=colors[i % len(colors)])

p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Show plot
show(p)
